# Langchain

### Prompts

- Get OpenAI key

In [ ]:
! pip install -r ../../requirements.txt

In [ ]:
import dotenv

# OPENAI_API_KEY=
dotenv.load_dotenv('.env')

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 'human', 'user', 'ai', 'assistant', or 'system'
prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful AI bot. Your name is {name}."),
            ("human", "Hello, how are you doing?"),
            ("ai", "I'm doing well, thanks!"),
            ("human", "{user_input}"),
        ])

final_prompt = prompt.invoke({
        "name": "Bob",
        "user_input": "What is your name?"
    })
final_prompt

In [ ]:
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot."),
    MessagesPlaceholder(variable_name="conversation", optional=True)
])

# Conversation can be retireved from other sources like Vector store, other APIs
final_prompt = prompt.invoke(
    {
        "conversation": [
            ("human", "Hi!"),
            ("ai", "How can I assist you today?"),
            ("human", "Can you make me an ice cream sundae?"),
            ("ai", "No.")
        ]
    }
)

final_prompt

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm.invoke("Hello, how are you doing?")

In [ ]:
# different model
llm = ChatOpenAI(model="gpt-4o")
llm.invoke("Hello, how are you doing?")

In [ ]:
# different Hyperparameters
print(ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=100).invoke("Hello, how are you doing?"))
print(ChatOpenAI(model="gpt-4o", temperature=2, max_tokens=100).invoke("Hello, how are you doing?"))

In [ ]:
### Different Output Parser
from langchain_core.output_parsers import StrOutputParser
chain = llm | StrOutputParser()
chain.invoke("Hello, how are you doing?")

# Chains

In [ ]:
prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful AI bot. Your name is {name}."),
            ("human", "Hello, how are you doing?"),
            ("ai", "I'm doing well, thanks!"),
            ("human", "{user_input}"),
        ])
llm = ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=100)
parser = StrOutputParser()

chain = prompt | llm | parser
chain.invoke({
        "name": "Bob",
        "user_input": "What is your name?"
})

In [ ]:
## Streaming invocation
stream = chain.stream({
        "name": "Bob",
        "user_input": "What is your name?"
})


for token in stream:
    print(token)

# saving and loading chain

In [ ]:
from langchain_core.load import dumpd
import json

json_str_chain = dumpd(chain)
with open("./chain.json", "w") as fp:
    json.dump(json_str_chain, fp, indent=4)

In [ ]:
chain.middle[0].lc_secrets

In [ ]:
from langchain_core.load import load
import os

with open("./chain.json", "r") as fp:
    chain = load(json.load(fp), secrets_map={"OPENAI_API_KEY": os.getenv("OPENAI_API_KEY")})
chain

In [ ]:
chain.invoke({
        "name": "Bob",
        "user_input": "What is your name?"
})